<h1 style="text-align: center; font-size: 36px; color: #3498db; font-weight: bold;">Prosody Tools</h1>

## <h2 style="text-align: center; font-size: 28px; color: #2ecc71; font-weight: bold;">Prosody Diarization Tool</h2>
### <h3 style="text-align: center; font-size: 24px; color: #e74c3c; font-family: 'Arial', sans-serif; font-weight: bold;">Stage 0</h3>


---
Project: Prosody

Institution: Reutlingen University

Author: Mohamad Eyad Alkostanini

Supervisor: Bakir Hadzic, Parvez Mohammed, (ViSiR)

Professor: Prof. Matthias Rätsch

Description:
This script performs diarization as part of the Prosody in Mental Health project.

--- 

# Building the folder structure

In [2]:
import os

# List of folder paths to create
folders = [
    r".\Database\0.Interviews\mp4",
    
    r".\Database\0.Interviews\wav\stage_0", # wav files for the first stage (first try to diarize the files)
    r".\Database\1.rttms\stage_0", # the diarization text files as '.rttm'
    r".\Database\2.Speaker_Parts\Stage_0", # the parts for each speaker
    r".\Database\3.Speaker_Combined\Stage_0", # the combined parts for each speaker
    
    r".\Database\0.Interviews\wav\stage_1",
    r".\Database\1.rttms\stage_1",
    r".\Database\2.Speaker_Parts\Stage_1",
    r".\Database\3.Speaker_Combined\Stage_1"
]

# Create folders if they do not exist
for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("All folders created successfully!")

# if the Intertviews are mp4 
# if the interviewws are wav 

All folders created successfully!


## convert mp4 to wav (if needed)

In [3]:
import os
from moviepy.editor import VideoFileClip

def convert_video_to_audio(input_video, output_audio):
    video_clip = VideoFileClip(input_video)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_audio, codec='pcm_s16le', bitrate='16k')

if __name__ == "__main__":
    input_dir = r".\Database\0.Interviews\mp4"
    output_dir = r".\Database\0.Interviews\wav\stage_0" #the wav files 

    if not os.path.exists(input_dir):
        os.makedirs(input_dir)
        
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for file in os.listdir(input_dir):
        if file.endswith(".mp4"):
            input_video = os.path.join(input_dir, file)
            output_audio = os.path.join(output_dir, os.path.splitext(file)[0] + ".wav")
            convert_video_to_audio(input_video, output_audio)

MoviePy - Writing audio in .\Database\0.Interviews\wav\stage_0\EXPRA_1_003.wav


MoviePy - Done.
MoviePy - Writing audio in .\Database\0.Interviews\wav\stage_0\EXPRA_3_012.wav


MoviePy - Done.


# Pyannote pipeline

In [4]:
from pyannote.audio import Pipeline
#import torch

# instantiate the pipeline
pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  use_auth_token="here you have to enter your auth-key") 

#https://huggingface.co/pyannote/speaker-diarization-3.1
#torch.cuda.is_available()

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
  if ismodule(module) and hasattr(module, '__file__'):



# Creating the RTTM files
## Stage 0, number of speakers are 2

In [5]:
# Save RTTMs

import os
import torchaudio
# diarization pipeline
diarization_pipeline = pipeline

# Path to WAV files
input_dir = r".\Database\0.Interviews\wav\stage_0"

# Create output if it doesn't exist
output_dir = r".\Database\1.rttms\stage_0"
os.makedirs(output_dir, exist_ok=True)

# Iterateall WAV files in the input
for file_name in os.listdir(input_dir):
    if file_name.endswith(".wav"):
        # Construct the full path to the WAV file
        wav_file_path = os.path.join(input_dir, file_name)

        # Perform diarization
        #waveform, sample_rate = torchaudio.load(wav_file_path)
        #diarization = pipeline({"waveform": waveform, "sample_rate": sample_rate})

        diarization = pipeline(wav_file_path, num_speakers=2) # number of speakers
 
        
        # Construct output RTTM file
        rttm_file_path = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}.rttm")

        # diarization output RTTM format
        with open(rttm_file_path, "w") as rttm_file:
            diarization.write_rttm(rttm_file)
            print (file_name, ' Done...')


  std = sequences.std(dim=-1, correction=1)



EXPRA_1_003.wav  Done...
EXPRA_3_012.wav  Done...


## Spliting the interview

In [6]:
import os
import pandas as pd

# directory RTTM files
rttm_dir = r".\Database\1.rttms\Stage_0"

columns = ["Type", "Interview_ID", "Channel", "Start", "Duration", "NA1", "NA2", "Speaker_ID", "NA3", "NA4"]

# combined data from all RTTM files
combined_df = pd.DataFrame(columns=columns)

# RTTM files in the directory
for file_name in os.listdir(rttm_dir):
    if file_name.endswith(".rttm"):
        # path to the RTTM file
        rttm_file_path = os.path.join(rttm_dir, file_name)

        # Read the RTTM file into a DataFrame
        rttm_df = pd.read_csv(rttm_file_path, sep=" ", header=None, names=columns)

        # Append RTTM file to the combined DataFrame
        combined_df = pd.concat([combined_df, rttm_df], ignore_index=True)

# Drop the columns 'NA1', 'NA2', 'NA3', and 'NA4' from the DataFrame
combined_df = combined_df.drop(columns=['NA1', 'NA2', 'NA3', 'NA4'])

combined_df


  combined_df = pd.concat([combined_df, rttm_df], ignore_index=True)



,Type,Interview_ID,Channel,Start,Duration,Speaker_ID
0,SPEAKER,EXPRA_1_003,1,2.377,7.763,SPEAKER_01
1,SPEAKER,EXPRA_1_003,1,10.966,3.510,SPEAKER_01
2,SPEAKER,EXPRA_1_003,1,15.539,0.253,SPEAKER_01
3,SPEAKER,EXPRA_1_003,1,17.277,1.789,SPEAKER_01
4,SPEAKER,EXPRA_1_003,1,21.327,2.869,SPEAKER_01
...,...,...,...,...,...,...
361,SPEAKER,EXPRA_3_012,1,663.556,9.096,SPEAKER_00
362,SPEAKER,EXPRA_3_012,1,670.525,0.321,SPEAKER_01
363,SPEAKER,EXPRA_3_012,1,673.057,7.239,SPEAKER_00
364,SPEAKER,EXPRA_3_012,1,680.937,0.658,SPEAKER_00


In [7]:
combined_df['Speaker_ID'].unique()

array(['SPEAKER_01', 'SPEAKER_00'], dtype=object)

In [8]:
combined_df['Interview_ID'].unique()

array(['EXPRA_1_003', 'EXPRA_3_012'], dtype=object)

In [9]:
# spliting

import os
import pandas as pd
from pydub import AudioSegment

# split based on start and duration
def split_audio(input_path, output_path, start, duration, filename):
    sound = AudioSegment.from_wav(input_path)
    split_sound = sound[start*1000:(start+duration)*1000]  # Pydub in milliseconds
    split_sound.export(os.path.join(output_path, filename), format="wav")

df = combined_df

input_dir = r".\Database\0.Interviews\wav\Stage_0"
output_dir = r".\Database\2.Speaker_Parts\Stage_0"

os.makedirs(output_dir, exist_ok=True)

i = 0
# each row in df
for index, row in df.iterrows():
    interview_id = row['Interview_ID']
    speaker_id = row['Speaker_ID']
    start = row['Start']
    duration = row['Duration']
    filename = f"{interview_id}_{speaker_id}_{i}.wav"
    input_path = os.path.join(input_dir, f"{interview_id}.wav")
    output_path = os.path.join(output_dir, speaker_id)
    os.makedirs(output_path, exist_ok=True)
    split_audio(input_path, output_path, start, duration, filename)
    i=i+1


# Join the Speakers

In [10]:
import os
import wave
from pydub import AudioSegment

# Function to concatenate wav files with a gap
def concatenate_wav_files_with_gap(directory, output, gap_duration=0.1):
    # Store wave file objects
    wave_files = {}

    # List files in directory
    for filename in os.listdir(directory):
        if filename.endswith(".wav"):
            # Prefix based on the first two parts of the filename
            if (filename.split('_')[0] == 'PBN'):
                prefix = '_'.join(filename.split('_')[:2])
            elif (filename.split('_')[0] == 'EXPRA'):
                prefix = '_'.join(filename.split('_')[:3])
            else:
                print('Name ERROR..............')
                continue

            # Append filename to the corresponding prefix list
            if prefix in wave_files:
                wave_files[prefix].append(filename)
            else:
                wave_files[prefix] = [filename]

    # Concatenate WAV files with the specified gap
    for prefix, files in wave_files.items():
        # Sort the files based on the last numeric part in the filename
        files.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

        # Create an empty AudioSegment to combine files
        combined = AudioSegment.silent(duration=0)  # Start with zero duration

        # Iterate through each file and concatenate with silence gaps
        for file in files:
            # Load each WAV file
            current_segment = AudioSegment.from_wav(os.path.join(directory, file))
            
            # Add a 0.2-second silent gap between segments
            combined += current_segment + AudioSegment.silent(duration=gap_duration * 1000)

        # Export the combined segment to the output directory
        output_filename = os.path.join(output, f"{prefix}_{os.path.basename(directory)}.wav")
        combined.export(output_filename, format="wav")
        print(f"Combined file saved: {output_filename}")

# Define directories and speakers
if __name__ == "__main__":
    speakers = [f"SPEAKER_{str(i).zfill(2)}" for i in range(2)]  # Adjust range as needed (e.g., range(10) for 10 speakers)
    stage = "Stage_0"
    
    for speaker in speakers:
        directory = rf".\Database\2.Speaker_Parts\{stage}\{speaker}"
        output = rf".\Database\3.Speaker_Combined\{stage}\{speaker}"
        # Create the output directory if it does not exist
        os.makedirs(output, exist_ok=True)

        print(f"Processing {speaker}...")
        # Run the concatenation function with the desired gap duration
        concatenate_wav_files_with_gap(directory, output, gap_duration=0.1)


Processing SPEAKER_00...
Combined file saved: .\Database\3.Speaker_Combined\Stage_0\SPEAKER_00\EXPRA_1_003_SPEAKER_00.wav
Combined file saved: .\Database\3.Speaker_Combined\Stage_0\SPEAKER_00\EXPRA_3_012_SPEAKER_00.wav
Processing SPEAKER_01...
Combined file saved: .\Database\3.Speaker_Combined\Stage_0\SPEAKER_01\EXPRA_1_003_SPEAKER_01.wav
Combined file saved: .\Database\3.Speaker_Combined\Stage_0\SPEAKER_01\EXPRA_3_012_SPEAKER_01.wav
